# Analysis.

In [16]:
import pandas as pd
from pandas import json_normalize

import requests 
import json
import os
from dotenv import load_dotenv #conda install -c conda-forge python-dotenv

import time
import importlib
import src.extraction as ex
importlib.reload(ex)

<module 'src.extraction' from 'd:\\ironhack\\proyectos\\PipeLine_proy2\\src\\extraction.py'>

## 1. USDA food database.
### 1.1. Exploration.

Did a research of all the USDA csv files and select only the more relevant for this proyect, they have a lot of information about comples dishes, lab tests, market prices, etc. 1. Searching for the right tables with value information for the proyect. 2. Cleaning each table, droping the columns and probably some rows with no need it information. ----try to merge it in python by changing the name of tha columns to the same in each. exported as csv 3. Merge the tables -- prbably on SQL exported again in csv and put it back on python.

#### 1.1.1. "Food csv".
Data frame of food, it has its id number and name of food.

In [6]:
url = "D:\ironhack\proyectos\PipeLine_proy2\\USDA_main_csv\\food.csv"

In [8]:
food = ex.import_csv (url)
food.sample()

,fdc_id,data_type,description,food_category_id,publication_date
982740,1555039,branded_food,"CHIPOTLE BAR-B-Q SAUCE, CHIPOTLE",NaN,2021-03-19


In [9]:
food.shape

(1185090, 5)

##### 1.1.1.1. Deleting.
Will remove the "food_category_id", "data_type" and the "publication_date" column and try to remove the index column, it appears that it does not do much, it doest have a column name for identification

In [10]:
food = ex.delete_food (food)
food

,fdc_id,description
0,1105904,WESSON Vegetable Oil 1 GAL
1,1105905,SWANSON BROTH BEEF
2,1105906,CAMPBELL'S SLOW KETTLE SOUP CLAM CHOWDER
3,1105907,CAMPBELL'S SLOW KETTLE SOUP CHEESE BROCCOLI
4,1105898,Discrepancy between the Atwater factor predict...
...,...,...
1185077,1757377,"MUSHROOMS, SHIITAKE"
1185078,1757378,"MUSHROOMS, SHIITAKE"
1185079,1757379,"MUSHROOMS, SHIITAKE"
1185080,1757380,"MUSHROOMS, SHIITAKE"


In [11]:
food.duplicated().sum()

0

#### 1.1.2. "Nutrients Table".
In this table can relate the data of nutrients and in wich food they are. they specifie the mount of the nutrient per 100g of food. Specified in unit defined in the nutrient table.

In [13]:
url2 = "D:\ironhack\proyectos\PipeLine_proy2\\USDA_main_csv\\nutrient.csv"
nutrient = ex.import_csv(url2)
nutrient.sample()

,id,name,unit_name,nutrient_nbr,rank
279,1283,Phytosterols,MG,636.0,15800.0


In [14]:
nutrient.shape

(463, 5)

##### 1.1.2.1. Deleting.
will delete must of the columns, except name and id, but will create a copy so i dont lose the information, could help me in the future.

In [18]:
nutrient = ex.delete_nutrient (nutrient)
nutrient.sample()

,id,name
18,1019,Pentosan


In [19]:
nutrient.shape

(463, 2)

#### 1.1.3. "Food_id Nutrient_id"
This table doesn´t appear in the information docs, but it loocks really helfull allready matching the nutrient with the food and the portion per 100g. Wont delete any column, but will start to change names of each Data Frame to matched them later. 

In [20]:
url3 = "D:\ironhack\proyectos\PipeLine_proy2\\USDA_main_csv\\JCS_FoodId-NutrientId-Per100g.csv"
ides = ex.import_csv (url3)
ides.sample()

,FoodId,NutrientId,Per100g
12422570,1561293,605,0.0


### 1.2. Merging.
Will merge all data frames into one, will chose the right tables for it and clean it more.

Looking for same elements in different tables at the ID rows, will change the names to matched them.

#### 1.2.1. Rename.

In [21]:
# change decription to food
# change fdc_id to food_id
food.rename(columns= {"fdc_id" : "food_id", "description" : "food"}, inplace=True)
food.sample()

,food_id,food
371186,712532,LEMON CAKE BAR


In [22]:
#change NutrientId to nutrient_id
#change foodId to food_id
ides.rename(columns= {"FoodId" : "food_id", "NutrientId" : "nutrient_id", 'Per100g ' : "Per100g"}, inplace=True)
ides.sample()

,food_id,nutrient_id,Per100g
4386864,624496,1258,0.0


In [23]:
# change id to nutrient_id
# change name to nutrient
nutrient.rename(columns= {"name" : "nutrient", "id" : "nutrient_id"}, inplace=True)
nutrient.sample()

,nutrient_id,nutrient
111,1113,25-hydroxycholecalciferol


#### 1.2.2. Merge.
Will do a inner merge, now with the same column name. also cleaned some other columns by doing drop, the id columns.

In [24]:
usda = ex.merge (nutrient, ides, food)
usda.sample()

,food,nutrient,Per100g
4352241,"TOMATO SAUCE, TOMATO","Fiber, total dietary",1.6


In [25]:
usda.shape

(7555511, 3)

In [28]:
usda.duplicated().sum()

2244337

In [29]:
usda.drop_duplicates(keep='first')

,food,nutrient,Per100g
0,Hummus,Nitrogen,1.28
1,Hummus,Total lipid (fat),19.00
2,Hummus,Ash,1.98
3,Hummus,Water,56.30
4,Hummus,Nitrogen,1.29
...,...,...,...
7555478,"OIL, OLIVE, EXTRA LIGHT",Delta-7-Stigmastenol,21.90
7555479,"OIL, PEANUT",Stigmastadiene,0.00
7555487,"OIL, SUNFLOWER",Stigmastadiene,0.00
7555495,"OIL, SAFFLOWER",Stigmastadiene,0.00


In [31]:
usda["nutrient"] = [i.strip().replace("  ", " ") for i in list(usda["nutrient"])]
usda["food"] = [i.strip().replace("  ", " ") for i in list(usda["food"])]

In [34]:
usda.drop_duplicates(subset=['food', 'nutrient'], keep='first', inplace= True)

In [35]:
usda.shape

(4731486, 3)

In [40]:
usda[["food", "nutrient", "Per100g"]].groupby(by=["food", "nutrient"]).agg({"nutrient":"counts"})

nutrient
food                                        nutrient                       nutrient                                
!AJUA!, CAFFEINE FREE SODA, MANDARIN ORANGE Carbohydrate, by difference    Carbohydrate, by difference            1
                                            Energy                         Energy                                 1
                                            Protein                        Protein                                1
                                            Sodium, Na                     Sodium, Na                             1
                                            Sugars, total including NLEA   Sugars, total including NLEA           1
...                                                                                                             ...
{TRAIL MIX}                                 Sodium, Na                     Sodium, Na                             1
                                            Sugars, total including NLEA   Sugars, total including NLEA           1
                                            Total lipid (fat)              Total lipid (fat)                      1
                                            Vitamin A, IU                  Vitamin A, IU                          1
                                            Vitamin C, total ascorbic acid Vitamin C, total ascorbic acid         1

[4731486 rows x 1 columns]